<a href="https://colab.research.google.com/github/Brikca/black-scholes-call-option/blob/main/Tugas_PMI_Opsi_Call_Black_Scholes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
from math import log, sqrt, exp
from scipy.stats import norm
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# =========================================================
# 1. Import Library
# =========================================================
import math
import numpy as np
import pandas as pd
import yfinance as yf
from scipy.stats import norm

# =========================================================
# 2. Fungsi bantu: volatilitas & Black-Scholes
# =========================================================
def annual_vol_from_yahoo(ticker):
    """
    Mengambil data 1 tahun terakhir dari Yahoo Finance
    dan menghitung volatilitas historis tahunan (σ).

    Rumus:
        r_t = ln(P_t / P_{t-1})
        σ_annual = std(r_t) * sqrt(252)
    """
    # Unduh data 1 tahun terakhir
    data = yf.download(ticker, period="1y", interval="1d", progress=False)
    data = data[['Close']].dropna()

    # Hitung return log harian
    data['log_return'] = np.log(data['Close'] / data['Close'].shift(1))
    data = data.dropna()

    # Hitung standar deviasi harian & annualisasi
    std_daily = data['log_return'].std()
    sigma_annual = std_daily * np.sqrt(252)

    # Pastikan harga terakhir bertipe float
    S0 = float(data['Close'].iloc[-1])

    return S0, sigma_annual, std_daily, len(data)


def black_scholes_call(S0, K, T_days, sigma, r):
    """
    Model Black-Scholes (tanpa dividen):

        C = S0 * N(d1) - K * exp(-rT) * N(d2)
        d1 = [ln(S0/K) + (r + 0.5σ²)T] / (σ√T)
        d2 = d1 - σ√T
    """
    T = T_days / 365  # ubah hari ke tahun

    d1 = (math.log(S0 / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)

    # Rumus harga call
    C = S0 * norm.cdf(d1) - K * math.exp(-r * T) * norm.cdf(d2)

    return C, T, d1, d2

# =========================================================
# 3. Parameter umum
# =========================================================
r = 0.0475      # suku bunga BI7DRR (Bank Indonesia) = 4.75%
T_days = 90     # waktu jatuh tempo sama (90 hari)

# 5 saham berbeda (A–E)
stocks = {
    "AAPL": 280,    # saham a - Apple
    "MSFT": 530,    # saham b - Microsoft
    "TSLA": 480,    # saham c - Tesla
    "AMZN": 250,    # saham d - Amazon
    "GOOGL": 300    # saham e - Alphabet
}

# =========================================================
# 4. Hitung opsi call untuk 5 saham
# =========================================================
hasil = []

for ticker, K in stocks.items():
    print(f"\n=== Memproses {ticker} ===")
    S0, sigma_annual, std_daily, n = annual_vol_from_yahoo(ticker)
    C, T, d1, d2 = black_scholes_call(S0, K, T_days, sigma_annual, r)

    hasil.append({
        "Saham": ticker,
        "S0 (Harga Saham Saat Ini)": round(S0, 2),
        "K (Strike Price)": K,
        "Waktu Jatuh Tempo (hari)": T_days,
        "Waktu Jatuh Tempo (tahun)": round(T, 4),
        "σ (Standar Deviasi Harian)": round(std_daily, 6),
        "σ (Volatilitas Tahunan)": round(sigma_annual, 4),
        "r (Suku Bunga BI)": r,
        "Harga Terakhir (Last Price)": round(S0, 2),
        "d1": round(d1, 4),
        "d2": round(d2, 4),
        "c (Harga Call - Black-Scholes)": round(C, 4)
    })

# =========================================================
# 5. Tabel hasil akhir
# =========================================================
df = pd.DataFrame(hasil)
pd.set_option("display.max_columns", None)

print("\n=== HASIL PERHITUNGAN OPSI CALL (MODEL BLACK-SCHOLES) ===")
display(df)


=== Memproses AAPL ===

=== Memproses MSFT ===

=== Memproses TSLA ===

=== Memproses AMZN ===

=== Memproses GOOGL ===

=== HASIL PERHITUNGAN OPSI CALL (MODEL BLACK-SCHOLES) ===


,Saham,S0 (Harga Saham Saat Ini),K (Strike Price),Waktu Jatuh Tempo (hari),Waktu Jatuh Tempo (tahun),σ (Standar Deviasi Harian),σ (Volatilitas Tahunan),r (Suku Bunga BI),Harga Terakhir (Last Price),d1,d2,c (Harga Call - Black-Scholes)
0,AAPL,270.37,280,90,0.2466,0.020419,0.3241,0.0475,270.37,-0.0642,-0.2251,14.5449
1,MSFT,517.81,530,90,0.2466,0.015130,0.2402,0.0475,517.81,-0.0373,-0.1565,21.8720
2,TSLA,456.56,480,90,0.2466,0.042560,0.6756,0.0475,456.56,0.0534,-0.2821,53.4850
3,AMZN,244.22,250,90,0.2466,0.021871,0.3472,0.0475,244.22,0.0185,-0.1539,15.4792
4,GOOGL,281.19,300,90,0.2466,0.020554,0.3263,0.0475,281.19,-0.2463,-0.4084,11.9797


# **Parameter**

| Parameter | Keterangan |
|------------|-------------|
| \(S_0\) | Harga saham saat ini (closing price terakhir dari Yahoo Finance) |
| \(K\) | Strike price (harga pelaksanaan opsi) |
| \(T\) | Waktu jatuh tempo (90 hari = 0.2466 tahun) |
| \(\sigma\) | Volatilitas tahunan, dihitung dari standar deviasi log-return harian × \(\sqrt{252}\) |
| \(r\) | Suku bunga bebas risiko (BI7DRR = 4.75%) |
| \(C\) | Harga call (hasil model Black–Scholes) |


# **Rumus yang Digunakan**

**1. Volalitas Historis**

$$
r_t = \ln\left(\frac{P_t}{P_{t-1}}\right)
$$

$$
\sigma_{\text{harian}} = \sqrt{\frac{1}{n-1}\sum_{t=1}^{n}(r_t - \bar{r})^2}
$$

$$
\sigma_{\text{tahunan}} = \sigma_{\text{harian}} \times \sqrt{252}
$$


**2. Model Black–Scholes (Call Eropa Tanpa Dividen)**
$$
C = S_0 \Phi(d_1) - K e^{-rT} \Phi(d_2)
$$

Dengan:
$$
d_1 = \frac{\ln(S_0/K) + (r + 0.5\sigma^2)T}{\sigma \sqrt{T}},
\qquad
d_2 = d_1 - \sigma \sqrt{T}
$$

Keterangan:
- $$(\Phi(d))$$ = fungsi distribusi kumulatif Normal Standar.  
- $$(e^{-rT})$$ = faktor diskonto dari risk-free rate.


# **Hasil Perhitungan**

| Saham | $(S_0)$ (Harga Saham) | $(K)$ | $(T)$ (hari) | $(T)$ (tahun) | $(sigma_{\text{harian}})$ | $(sigma_{\text{tahunan}})$ | $(r)$ | $(d_1)$ | $(d_2)$ | $(C)$ (Harga Call) |
|--------|------------------|---|-----------|------------|-------------|--------------|---|----|----|----------------|
| **AAPL** | 270.37 | 280 | 90 | 0.2466 | 0.020419 | 0.3241 | 0.0475 | -0.0642 | -0.2251 | **14.5449** |
| **MSFT** | 517.81 | 530 | 90 | 0.2466 | 0.015130 | 0.2402 | 0.0475 | -0.0373 | -0.1565 | **21.8720** |
| **TSLA** | 456.56 | 480 | 90 | 0.2466 | 0.042560 | 0.6756 | 0.0475 | 0.0534 | -0.2821 | **53.4850** |
| **AMZN** | 244.22 | 250 | 90 | 0.2466 | 0.021871 | 0.3472 | 0.0475 | 0.0185 | -0.1539 | **15.4792** |
| **GOOGL** | 281.19 | 300 | 90 | 0.2466 | 0.020554 | 0.3263 | 0.0475 | -0.2463 | -0.4084 | **11.9797** |


## Interpretasi Hasil

1. **Moneyness ($(S_0)$ vs $(K)$):**  
   Semua saham berada dalam kondisi *Out-of-the-Money (OTM)* karena $(S_0 < K)$.  
   Artinya opsi baru akan bernilai positif jika harga saham naik melebihi strike price sebelum jatuh tempo.

2. **Peran Volatilitas (\(\sigma\)):**  
   Semakin tinggi volatilitas, semakin besar harga opsi.  
   - **TSLA** memiliki volatilitas tahunan tertinggi (\(0.6756\)) → harga call paling mahal (**53.4850 USD**).  
   - **GOOGL** volatilitas rendah dan paling OTM → harga call terendah (**11.9797 USD**).

3. **Peran Suku Bunga (\(r\)):**  
   Karena \(r = 4.75\%\) konstan untuk semua saham, efeknya kecil terhadap perbedaan antar-saham.  
   Nilai \(r\) hanya memengaruhi faktor diskonto \(e^{-rT}\).

4. **Peran Waktu (\(T\)):**  
   Semua saham memakai \(T = 0.2466\) tahun (90 hari), sehingga perbedaan harga murni berasal dari moneyness dan volatilitas.


## Kesimpulan

- Model Black–Scholes berhasil menghitung harga opsi call dari **5 saham berbeda (A–E)** dengan volatilitas historis 1 tahun.  
- Semakin **tinggi volatilitas (\(\sigma\))** dan semakin **dekat \(S_0\) ke \(K\)**, maka **harga opsi call (\(C\))** semakin tinggi.  
- **TSLA** memiliki risiko tertinggi (\(\sigma = 0.6756\)) → opsi paling mahal (**53.4850 USD**).  
- **GOOGL** paling murah karena paling jauh dari strike dan volatilitasnya rendah.  
- Semua perhitungan menggunakan **suku bunga bebas risiko Bank Indonesia \(r = 4.75\%\)** dan **\(T = 90\) hari**.
